# Pasos:

1. Particionar datos de Entrenamiento (80%) y Pruebas(20%) 
2. Entrenar Modelos a evaluar:
    - Redes Neuronales
    - XGBoost
    - Naive Bayes
    - SVM
    - Ensamble Modelo (RandomForest o Arbol de decisión)
3. Evaluar los hiperparametros (RandomSearchCV)
4. Entrenar modelos con mejores hiperparámetros encontrados en 3.
5. Evaluar modelos con data de pruebas (Accuracy)

## Particionar datos de Entrenamiento (80%) y Pruebas(20%)

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
data = "./data/02_data_intermedia/MI_preprocesado.csv"

In [3]:
dataProcesada = pd.read_csv(data)

In [4]:
dataProcesada.head()

,Unnamed: 0,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,...,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS,COMPLICATION_OUT
0,0,77.0,1,2.0,1.0,1.0,2.0,3.0,0.0,7.0,...,0,0,0,0,0,0,0,0,0,0
1,1,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2,52.0,1,0.0,0.0,0.0,2.0,2.0,0.0,2.0,...,0,0,0,0,0,0,0,0,0,0
3,3,68.0,0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,...,0,0,0,0,0,1,0,0,0,0
4,4,60.0,1,0.0,0.0,0.0,2.0,3.0,0.0,7.0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
len(dataProcesada)

1700

In [7]:
from sklearn import preprocessing, neighbors
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

In [8]:
train, test = train_test_split(dataProcesada, test_size=0.2)

In [9]:
len(train)

36

In [10]:
len(test)

9